<a href="https://colab.research.google.com/github/yuce-8/alpha/blob/main/Custom_Data_Loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YUCE-8.tech API - CUSTOM DATA LOADER
**THIS IS FOR EDUCATIONAL / RESEARCH & DEVELOPMENT PURPOSES ONLY!**


In [51]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install requests

import json
import requests


INTERVAL_1D = '1d'
INTERVAL_1HOUR = '1hour'
INTERVAL_4HOURS = '4hour'
INTERVAL_30MIN = '30min'
INTERVAL_15MIN = '15min'
INTERVAL_5MIN = '5min'
INTERVAL_1MIN = '1min'


license_key = 'default'
user_name = 'user1@abc.com'
symbol = 'custom_symbol_123'
interval = INTERVAL_4HOURS

## Connect to YUCE-8
this is subject to change in the next versions

In [52]:
class GCF():
  def __init__(self):
    self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/custom_data_loader'
    print('YUCE-8 CUSTOM DATA LOADER ready @ ', self.url)

  def push(self, json_obj, return_result=False):
    r = requests.post(self.url, json=json_obj)
    if return_result:
      return r

yuce_8 = GCF()

YUCE-8 CUSTOM DATA LOADER ready @  https://us-central1-yuce-8-v1.cloudfunctions.net/custom_data_loader


## REMOVE ALL EXISTING ENTRIES

In [53]:
def remove_all_entries(user_name, symbol, interval):
  global license_key
  req = yuce_8.push({
      'action': 'remove_all_entries',
      'key': license_key,
      'user_name': user_name,
      'symbol': symbol,
      'interval': interval
  }, return_result=True)
  if req.status_code == 200:
    return req.json()['result']
  else:
    return ';-( ' + req.json()['error']


remove_all_entries(user_name, symbol, interval)

## ATTACH CANDLES 
the web service will automatically detect existing entries (by date)

In [54]:
def attach_candles(user_name, symbol, interval, candles):
  global license_key
  req = yuce_8.push({
      'action': 'attach_candles',
      'key': license_key,
      'user_name': user_name,
      'symbol': symbol,
      'interval': interval,
      'candles': candles
  }, return_result=True)
  if req.status_code == 200:
    return req.json()['result']
  else:
    return ';-( ' + req.json()['error']


all_candles = []
def add_candle(timestamp, open_, high, low, close, adj_close, volume):
    global all_candles
    all_candles.append({'Date': timestamp, 'Open': open_, 'High': high, 'Low': low, 'Close': close, 'Adj Close': adj_close, 'Volume': volume})

add_candle('2021-04-01 20:00:00', 10, 12, 10, 10, 10, 100)

print(attach_candles(user_name, symbol, interval, all_candles), ' candles attached to custom database')

1  candles attached to custom database


**ALTERNATIVELY** upload a complete CSV file for the given *INTERVAL*

In [55]:
from google.colab import files
uploaded = files.upload()

Saving NIFTYPE1210000N.csv to NIFTYPE1210000N (1).csv


transfer the csv into a pandas and diplay it (for verification purposes)

In [56]:
import io
import pandas as pd
file_name = list(uploaded.keys())[0]
print('processing ', file_name)
df = pd.read_csv(io.BytesIO(uploaded[file_name]))
df

processing  NIFTYPE1210000N.csv


,DateTime,O,H,L,C,Vol
0,2020-02-01 09:20:00,305.00,342.50,305.00,313.95,29175
1,2020-02-01 09:25:00,315.45,323.10,304.55,306.55,12000
2,2020-02-01 09:30:00,306.60,307.15,298.75,303.60,7875
3,2020-02-01 09:35:00,301.80,303.30,290.75,295.50,12900
4,2020-02-01 09:40:00,295.70,303.50,294.80,299.75,10050
...,...,...,...,...,...,...
1486,2020-02-28 15:10:00,982.05,995.00,982.05,993.00,1350
1487,2020-02-28 15:15:00,990.00,990.00,965.00,965.00,2700
1488,2020-02-28 15:20:00,950.00,950.00,940.00,940.00,375
1489,2020-02-28 15:25:00,955.00,961.20,950.25,950.25,825


now upload this dataframe to YUCE-8

In [58]:
chunk_max = 500
chunk_counter = 0

all_candles = []
for index, row in df.iterrows():

  add_candle(str(row[0]), row[1], row[2], row[3], row[4], row[4], row[5])  
  chunk_counter += 1
  if chunk_counter >= chunk_max:
    print(attach_candles(user_name, symbol, interval, all_candles), ' candles attached to custom database')
    all_candles = []
    chunk_counter = 0

# any candles left?
if len(all_candles) > 0:
    print(attach_candles(user_name, symbol, interval, all_candles), ' candles attached to custom database')


0  candles attached to custom database
500  candles attached to custom database
491  candles attached to custom database


## GET FIRST AND LAST TIMESTAMP OF EXISTING ENTRIES 
the checks the custom database and returns the first and last date of all existing entries

In [59]:
def get_first_and_last_entry(user_name, symbol, interval):
  global license_key
  req = yuce_8.push({
      'action': 'get_first_and_last_entry',
      'key': license_key,
      'user_name': user_name,
      'symbol': symbol,
      'interval': interval
  }, return_result=True)
  if req.status_code == 200:
    return req.json()['result']
  else:
    return ';-( ' + req.json()['error']


print(get_first_and_last_entry(user_name, symbol, interval))

['2020-02-01 09:20:00', '2021-04-01 20:00:00']


## GET DATES OF ALL  EXISTING ENTRIES

In [60]:
def get_all_dates(user_name, symbol, interval):
  global license_key
  req = yuce_8.push({
      'action': 'get_all_dates',
      'key': license_key,
      'user_name': user_name,
      'symbol': symbol,
      'interval': interval
  }, return_result=True)
  if req.status_code == 200:
    return req.json()['result']
  else:
    return ';-( ' + req.json()['error']


print(get_all_dates(user_name, symbol, interval))

['2020-02-01 09:20:00', '2020-02-01 09:25:00', '2020-02-01 09:30:00', '2020-02-01 09:35:00', '2020-02-01 09:40:00', '2020-02-01 09:45:00', '2020-02-01 09:50:00', '2020-02-01 09:55:00', '2020-02-01 10:00:00', '2020-02-01 10:05:00', '2020-02-01 10:10:00', '2020-02-01 10:15:00', '2020-02-01 10:20:00', '2020-02-01 10:25:00', '2020-02-01 10:30:00', '2020-02-01 10:35:00', '2020-02-01 10:40:00', '2020-02-01 10:45:00', '2020-02-01 10:50:00', '2020-02-01 10:55:00', '2020-02-01 11:00:00', '2020-02-01 11:05:00', '2020-02-01 11:10:00', '2020-02-01 11:15:00', '2020-02-01 11:20:00', '2020-02-01 11:25:00', '2020-02-01 11:30:00', '2020-02-01 11:35:00', '2020-02-01 11:40:00', '2020-02-01 11:45:00', '2020-02-01 11:50:00', '2020-02-01 11:55:00', '2020-02-01 12:00:00', '2020-02-01 12:05:00', '2020-02-01 12:10:00', '2020-02-01 12:15:00', '2020-02-01 12:20:00', '2020-02-01 12:25:00', '2020-02-01 12:30:00', '2020-02-01 12:35:00', '2020-02-01 12:40:00', '2020-02-01 12:45:00', '2020-02-01 12:50:00', '2020-02-0

# DISCLAIMER

**THIS IS FOR EDUCATIONAL / RESEARCH & DEVELOPMENT PURPOSES ONLY!**


The information on this site is provided by YUCE-8 and the author and it is not to be construed as an offer or solicitation for the purchase or sale of any financial instrument or the provision of an offer to provide investment services. Information, opinions and comments contained on this site are not under the scope of investment advisory services. Investment advisory services are given according to the investment advisory contract, signed between the intermediary institutions, portfolio management companies, investment banks and the clients. Opinions and comments contained in this site reflect the personal views of the analysts / AI algorithms who supplied them. The investments discussed or recommended in this report may involve significant risk, may be illiquid and may not be suitable for all investors. Therefore, making decisions with respect to the information in this report may cause inappropriate results and a total loss.
Futures and forex trading contains substantial risk and is not for every investor. An investor could potentially lose all or more than the initial investment. Risk capital is money that can be lost without jeopardizing ones’ financial security or life style. Only risk capital should be used for trading and only those with sufficient risk capital should consider trading. Past performance is not necessarily indicative of future results
Hypothetical performance results have many inherent limitations, some of which are described below. no representation is being made that any account will or is likely to achieve profits or losses similar to those shown; in fact, there are frequently sharp differences between hypothetical performance results and the actual results subsequently achieved by any particular trading program. One of the limitations of hypothetical performance results is that they are generally prepared with the benefit of hindsight. In addition, hypothetical trading does not involve financial risk, and no hypothetical trading record can completely account for the impact of financial risk of actual trading. for example, the ability to withstand losses or to adhere to a particular trading program in spite of trading losses are material points which can also adversely affect actual trading results. There are numerous other factors related to the markets in general or to the implementation of any specific trading program which cannot be fully accounted for in the preparation of hypothetical performance results and all which can adversely affect trading results.
Testimonials appearing on this website may not be representative of other clients or customers and is not a guarantee of future performance or success.



No Investment Advice & No Reliance
YUCE-8 IS A DATA ANALYTICS SERVICE. YUCE-8 IS NO BROKER/DEALER, WE ARE NOT AN INVESTMENT ADVISOR, WE HAVE NO ACCESS TO NON-PUBLIC INFORMATION ABOUT PUBLICLY TRADED COMPANIES, AND THIS IS NOT A PLACE FOR THE GIVING OR RECEIVING OF FINANCIAL ADVICE, ADVICE CONCERNING INVESTMENT DECISIONS OR TAX OR LEGAL ADVICE. WE ARE NOT REGULATED BY THE SEC OR ANY FINANCIAL SERVICES AUTHORITY. 
ACCORDINGLY, YUCE-8 WILL NOT BE LIABLE, WHETHER IN CONTRACT, TORT (INCLUDING NEGLIGENCE) OR OTHERWISE, IN RESPECT OF ANY DAMAGE, EXPENSE OR OTHER LOSS YOU MAY SUFFER ARISING OUT OF SUCH INFORMATION OR ANY RELIANCE YOU MAY PLACE UPON SUCH INFORMATION. ANY ARRANGEMENTS BETWEEN YOU AND ANY THIRD PARTY CONTACTED VIA THE WEBSITE ARE AT YOUR SOLE RISK.